In [3]:
import pandas as pd

fulldata = pd.read_csv('C:/SEMESTER 5/STKI/Tugas 3 STKI/preprocessing/fulldata.csv')

C:\Users\isaia\AppData\Local\Temp\ipykernel_76780\2031201950.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  fulldata = pd.read_csv('C:/SEMESTER 5/STKI/Tugas 3 STKI/preprocessing/fulldata.csv')


In [5]:
import numpy as np

data = fulldata.copy()
data["user_rating"].replace(to_replace = -1 , value = np.nan ,inplace=True)
data = data.dropna(axis = 0)
print("Null values after final pre-processing :")
data.isna().sum().to_frame().T.style.set_properties()

C:\Users\isaia\AppData\Local\Temp\ipykernel_76780\2882349054.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["user_rating"].replace(to_replace = -1 , value = np.nan ,inplace=True)


Null values after final pre-processing :


,anime_id,name,genre,type,episodes,rating,members,user_id,user_rating
0,0,0,0,0,0,0,0,0,0


Ada banyak pengguna yang hanya memberi rating sekali, bahkan jika mereka telah memberi rating 5 anime, hal itu tidak dapat dianggap sebagai catatan yang berharga untuk rekomendasi. Jadi, kami akan mempertimbangkan minimal 50 peringkat oleh pengguna sebagai nilai ambang batas.

In [6]:
selected_users = data["user_id"].value_counts()
data = data[data["user_id"].isin(selected_users[selected_users >= 50].index)]

Kita akan membuat tabel pivot yang terdiri dari baris sebagai judul dan kolom sebagai id pengguna, hal ini akan membantu kita dalam membuat matriks jarang yang dapat sangat membantu dalam mencari kemiripan cosinus.

In [7]:
data_pivot_temp = data.pivot_table(index="name",columns="user_id",values="user_rating").fillna(0)
data_pivot_temp.head()

user_id,3,5,7,11,14,17,21,23,24,27,...,73495,73499,73500,73501,73502,73503,73504,73507,73510,73515
name,,,,,,,,,,,,,,,,,,,,,
&quot;0&quot;,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Memoire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
&quot;Bungaku Shoujo&quot; Movie,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
&quot;Eiji&quot;,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Kami memiliki banyak simbol karakter Jepang atau karakter khusus dalam nama anime. Mari kita hapus semua itu dengan menggunakan fungsi ini

In [8]:
import re
def text_cleaning(text):
    text = re.sub(r'&quot;', '', text)
    text = re.sub(r'.hack//', '', text)
    text = re.sub(r'&#039;', '', text)
    text = re.sub(r'A&#039;s', '', text)
    text = re.sub(r'I&#039;', 'I\'', text)
    text = re.sub(r'&amp;', 'and', text)
    
    return text

data["name"] = data["name"].apply(text_cleaning)

In [9]:
data_pivot = data.pivot_table(index="name",columns="user_id",values="user_rating").fillna(0)
print("After Cleaning the animes names, let's see how it looks like.")
data_pivot.head()

After Cleaning the animes names, let's see how it looks like.


user_id,3,5,7,11,14,17,21,23,24,27,...,73495,73499,73500,73501,73502,73503,73504,73507,73510,73515
name,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009 Re:Cyborg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009-1: RandB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Pengembangan Model Rekomendasi

***Teknik Matrix Factorization atau KNN***


Collaborative Filtering adalah teknik yang dapat menyaring item yang mungkin disukai pengguna berdasarkan reaksi dari pengguna yang serupa. Teknik ini bekerja dengan mencari sekelompok besar orang dan menemukan sekumpulan kecil pengguna yang memiliki selera yang mirip dengan pengguna tertentu. Kami akan menggunakan Cosine similarity yang merupakan metrik yang digunakan untuk mengukur seberapa mirip dokumen terlepas dari ukurannya. Secara matematis, metrik ini mengukur kosinus sudut antara dua vektor yang diproyeksikan dalam ruang multi-dimensi. Kemiripan kosinus menguntungkan karena meskipun dua dokumen yang mirip berjauhan menurut jarak Euclidean (karena ukuran dokumen), kemungkinan besar dokumen-dokumen tersebut masih berorientasi lebih dekat satu sama lain. Semakin kecil sudutnya, semakin tinggi kemiripan kosinusnya.

In [10]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

data_matrix = csr_matrix(data_pivot.values)

model_knn = NearestNeighbors(metric = "cosine", algorithm = "brute")
model_knn.fit(data_matrix)

query_no = np.random.choice(data_pivot.shape[0]) # random anime title and finding recommendation
print(f"We will find recommendation for {query_no} no anime which is {data_pivot.index[query_no]}.")
distances, indices = model_knn.kneighbors(data_pivot.iloc[query_no,:].values.reshape(1, -1), n_neighbors = 6)

We will find recommendation for 8519 no anime which is Tasukeai no Rekishi: Inochi Hoken no Hajimari.


In [12]:
anime = pd.read_csv('C:/SEMESTER 5/STKI/Tugas 3 STKI/preprocessing/anime.csv')
no = []
name = []
distance = []
rating = []

for i in range(0, len(distances.flatten())):
    if i == 0:
        print(f"Recommendations for {data_pivot.index[query_no]} viewers :/n")
    else:
        #  print(f"{i}: {data_pivot.index[indices.flatten()[i]]} , with distance of {distances.flatten()[i]}")        
        no.append(i)
        name.append(data_pivot.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])
        rating.append(*anime[anime["name"]==data_pivot.index[indices.flatten()[i]]]["rating"].values)

dic = {"No" : no, "Anime Name" : name, "Rating" : rating}
recommendation = pd.DataFrame(data = dic)
recommendation.set_index("No", inplace = True)
recommendation.style.set_properties()

Recommendations for Tasukeai no Rekishi: Inochi Hoken no Hajimari viewers :/n


,Anime Name,Rating
No,,
1,Kuroi Kikori to Shiroi Kikori,6.200000
2,Kappa Kawatarou,4.850000
3,Yumemi Douji,5.270000
4,Plus 50000-nen,5.080000
5,Nezumi no Yomeiri,4.960000


## Evaluate


Content Based Filtering merekomendasikan item berdasarkan perbandingan antara konten item dan profil pengguna. Konten setiap item direpresentasikan sebagai sekumpulan deskriptor atau istilah, biasanya berupa kata-kata yang muncul dalam dokumen. Rekomendasi berbasis konten bekerja dengan data yang diberikan pengguna, baik secara eksplisit (peringkat) atau implisit (mengklik tautan). Berdasarkan data tersebut, profil pengguna dibuat yang kemudian digunakan untuk memberikan saran kepada pengguna. Ketika pengguna memberikan lebih banyak masukan atau melakukan tindakan atas rekomendasi, mesin akan semakin akurat.

**Term Frequency(TF) & Inverse Document Frequency(IDF)**

TF adalah frekuensi sebuah kata dalam sebuah dokumen. IDF adalah kebalikan dari frekuensi dokumen di antara seluruh korpus dokumen. TF-IDF digunakan terutama karena dua alasan: Misalkan kita mencari “kebangkitan analitik” di Google. Sudah pasti bahwa “the” akan muncul lebih sering daripada “analytics”, tetapi kepentingan relatif dari analytics lebih tinggi daripada sudut pandang kueri penelusuran. Dalam kasus seperti itu, pembobotan TF-IDF meniadakan efek dari kata-kata berfrekuensi tinggi dalam menentukan tingkat kepentingan suatu item (dokumen).
Di sini kita akan menggunakannya pada genre anime sehingga kita dapat merekomendasikan konten kepada pengguna berdasarkan genre.


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents="unicode", analyzer="word",
                      token_pattern=r"\w{1,}", ngram_range=(1, 3), stop_words = "english")

rec_data = fulldata.copy()
rec_data.drop_duplicates(subset ="name", keep = "first", inplace = True)
rec_data.reset_index(drop = True, inplace = True)
genres = rec_data["genre"].str.split(", | , | ,").astype(str)
tfv_matrix = tfv.fit_transform(genres)


Selain itu, scikit-learn sudah menyediakan metrik berpasangan (alias kernel dalam bahasa machine learning) yang dapat digunakan untuk representasi koleksi vektor yang padat maupun yang jarang. Di sini kita perlu menetapkan nilai 1 untuk anime yang direkomendasikan dan 0 untuk anime yang tidak direkomendasikan. Kita akan menggunakan kernel sigmoid di sini.

In [14]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig = sigmoid_kernel(tfv_matrix, tfv_matrix)      # Computing sigmoid kernel

rec_indices = pd.Series(rec_data.index, index = rec_data["name"]).drop_duplicates()


# Recommendation Function
def give_recommendation(title, sig = sig):
    
    idx = rec_indices[title] # Getting index corresponding to original_title

    sig_score = list(enumerate(sig[idx]))  # Getting pairwsie similarity scores 
    sig_score = sorted(sig_score, key=lambda x: x[1], reverse=True)
    sig_score = sig_score[1:11]
    anime_indices = [i[0] for i in sig_score]
     
    # Top 10 most similar movies
    rec_dic = {"No" : range(1,11), 
               "Anime Name" : anime["name"].iloc[anime_indices].values,
               "Rating" : anime["rating"].iloc[anime_indices].values,
               "Genre" : anime["genre"].iloc[anime_indices].values}
    dataframe = pd.DataFrame(data = rec_dic)
    dataframe.set_index("No", inplace = True)
    
    print(f"Recommendations for {title} viewers :\n")
    
    return dataframe.style.set_properties()

In [15]:
give_recommendation("Naruto")

Recommendations for Naruto viewers :



,Anime Name,Rating,Genre
No,,,
1,To LOVE-Ru Darkness OVA,7.820000,"Comedy, Ecchi, Harem, Romance, School, Sci-Fi, Shounen"
2,Hanbun no Tsuki ga Noboru Sora,7.690000,"Comedy, Drama, Romance"
3,Mai-HiME,7.590000,"Action, Comedy, Drama, Fantasy, Magic, Mecha, Romance, School, Shoujo Ai"
4,Doraemon Movie 28: Nobita to Midori no Kyojin Den,7.540000,"Adventure, Comedy, Fantasy, Kids, Shounen"
5,Rurouni Kenshin Special,7.510000,"Action, Historical, Romance, Samurai"
6,Pikmin Short Movies,7.270000,"Fantasy, Kids"
7,Croket!,7.120000,"Action, Adventure, Comedy, Fantasy"
8,Omoide no Album,5.140000,"Kids, Music, School, Slice of Life"
9,Fukashigi no Kazoekata,5.370000,Kids


In [16]:
give_recommendation("To LOVE-Ru Darkness OVA")

Recommendations for To LOVE-Ru Darkness OVA viewers :



,Anime Name,Rating,Genre
No,,,
1,Kyoukai no Kanata Movie: I'll Be Here - Kako-hen - Yakusoku no Kizuna,7.920000,Music
2,Tetsuwan Birdy Decode:02,7.830000,"Action, Comedy, Sci-Fi"
3,Bishoujo Senshi Sailor Moon Crystal Season III,7.830000,"Demons, Magic, Romance, Shoujo"
4,Mobile Suit Gundam: Char's Counterattack,7.810000,"Drama, Mecha, Military, Sci-Fi, Space"
5,Suzuka,7.350000,"Drama, Romance, Sports"
6,Uchuu Patrol Luluco,7.700000,"Action, Adventure, Comedy, Space"
7,Date A Live,7.540000,"Comedy, Harem, Mecha, Romance, School, Sci-Fi"
8,Yumekui Merry,7.080000,"Action, Shounen, Supernatural"
9,The iDOLM@STER Cinderella Girls 2nd Anniversary PV,7.060000,Music


In [17]:
give_recommendation("Mogura no Motoro")

Recommendations for Mogura no Motoro viewers :



,Anime Name,Rating,Genre
No,,,
1,Cat's Eye,7.210000,"Action, Adventure, Comedy, Mystery, Romance"
2,Bakusou Kyoudai Let's & Go!! WGP Bousou Mini Yonku Daitsuiseki,6.820000,"Adventure, Cars, Shounen, Sports"
3,Je T'aime,6.790000,Sci-Fi
4,Pokemon Ranger: Hikari no Kiseki,6.680000,"Adventure, Comedy, Fantasy, Kids"
5,Yuki,6.670000,"Adventure, Demons, Fantasy"
6,Pokemon: Pikachu Tankentai,6.650000,"Adventure, Comedy, Fantasy, Kids"
7,Nanami-chan 2nd Series,6.610000,"Comedy, Kids"
8,Sengoku Collection,6.540000,"Fantasy, Parody, Samurai"
9,Hoero! Bun Bun,6.530000,"Adventure, Slice of Life"
